In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv("../dropped_unnecessary.csv")
print(df.head())

         date  dif  draw_count  goals_against  goals_scored  \
0  2010-08-20    2           0              1             3   
1  2010-10-09    3           0              3             6   
2  2010-09-21   -2           1             10             8   
3  2010-01-10   -4           1             13             9   
4  2010-10-22   -8           1             18            10   

               league  lose_count  points  ranking  season  ...  \
0  Germany Bundesliga           0       3        2    2010  ...   
1  Germany Bundesliga           1       6        5    2010  ...   
2  Germany Bundesliga           2       7       10    2010  ...   
3  Germany Bundesliga           4       7       14    2010  ...   
4  Germany Bundesliga           6       7       15    2010  ...   

  Team Width_away  Defender Line_away  DP_away  IP_away  SAA_away  TAA_away  \
0            Wide               Cover     13.0     10.0      29.0     27.45   
1          Normal               Cover     10.0      8.0     

In [19]:
print(df.columns)

Index(['date', 'dif', 'draw_count', 'goals_against', 'goals_scored', 'league',
       'lose_count', 'points', 'ranking', 'season', 'team_home', 'week',
       'win_count', 'ID', 'OVA', 'ATT', 'MID', 'DEF', 'Transfer Budget',
       'Speed', 'Dribbling', 'BuildPassing', 'BuildPositioning', 'Crossing',
       'ChancePassing', 'Shooting', 'ChancePositioning', 'Aggression',
       'Pressure', 'Team Width', 'Defender Line', 'DP', 'IP', 'SAA', 'TAA',
       'away_score_final', 'away_score_first', 'home_score_final',
       'home_score_first', 'dif_away', 'draw_count_away', 'goals_against_away',
       'goals_scored_away', 'lose_count_away', 'points_away', 'ranking_away',
       'team_away', 'win_count_away', 'ID_away', 'OVA_away', 'ATT_away',
       'MID_away', 'DEF_away', 'Transfer Budget_away', 'Speed_away',
       'Dribbling_away', 'BuildPassing_away', 'BuildPositioning_away',
       'Crossing_away', 'ChancePassing_away', 'Shooting_away',
       'ChancePositioning_away', 'Aggression_away'

In [20]:
df.drop(columns=["date", "league", "season", "week", "win", "draw", "lose", "ID", "ID_away", "team_home", "team_away"], inplace=True)

,dif,draw_count,goals_against,goals_scored,lose_count,points,ranking,win_count,OVA,ATT,...,ChancePositioning_away,Aggression_away,Pressure_away,Team Width_away,Defender Line_away,DP_away,IP_away,SAA_away,TAA_away,outcome_of_match
0,2,0,1,3,0,3,2,1,70.0,70.0,...,Normal,Press,Medium,Wide,Cover,13.0,10.0,29.0,27.45,-1
1,3,0,3,6,1,6,5,2,70.0,70.0,...,Normal,Press,Medium,Normal,Cover,10.0,8.0,30.0,27.36,1
2,-2,1,10,8,2,7,10,2,70.0,70.0,...,Normal,Double,Medium,Normal,Cover,16.0,14.0,31.0,24.09,1
3,-4,1,13,9,4,7,14,2,70.0,70.0,...,Normal,Press,High,Normal,Cover,17.0,15.0,32.0,27.18,1
4,-8,1,18,10,6,7,15,2,70.0,70.0,...,Normal,Press,Medium,Normal,Cover,10.0,7.0,28.0,26.18,1
